<a href="https://colab.research.google.com/github/Shiyx27/ForestFireDetection/blob/main/ForestFireDetection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:

# IMPORTANT: RUN THIS CELL IN ORDER TO IMPORT YOUR KAGGLE DATA SOURCES
# TO THE CORRECT LOCATION (/kaggle/input) IN YOUR NOTEBOOK,
# THEN FEEL FREE TO DELETE THIS CELL.
# NOTE: THIS NOTEBOOK ENVIRONMENT DIFFERS FROM KAGGLE'S PYTHON
# ENVIRONMENT SO THERE MAY BE MISSING LIBRARIES USED BY YOUR
# NOTEBOOK.

import os
import sys
from tempfile import NamedTemporaryFile
from urllib.request import urlopen
from urllib.parse import unquote, urlparse
from urllib.error import HTTPError
from zipfile import ZipFile
import tarfile
import shutil

CHUNK_SIZE = 40960
DATA_SOURCE_MAPPING = 'forest-fire-smoke-and-non-fire-image-dataset:https%3A%2F%2Fstorage.googleapis.com%2Fkaggle-data-sets%2F3135864%2F5721181%2Fbundle%2Farchive.zip%3FX-Goog-Algorithm%3DGOOG4-RSA-SHA256%26X-Goog-Credential%3Dgcp-kaggle-com%2540kaggle-161607.iam.gserviceaccount.com%252F20240315%252Fauto%252Fstorage%252Fgoog4_request%26X-Goog-Date%3D20240315T182252Z%26X-Goog-Expires%3D259200%26X-Goog-SignedHeaders%3Dhost%26X-Goog-Signature%3D748f5a96cc50bd06fb75310df19cae641c1069a0124e8c547f1beccfe00ec972909858341c977653a46866a060b0d14021917920ac3264bac8e968b763875da15fc0b76199001309750b9ca37f40b2c47de4c5847de6c9df02e2f9eb8c47bf0064e262aebe2641b087253bc373647c1b1f38f182b1c7dcbfbf01322ce72f1db65c2ffa1eaa21c818b44117b5eccac65e202607a4e832604936d514aade9d6c57e066d54ffd984ede29975b9e376e97162653e26827a6faa5365413b515ff67e80ebe651466f1129d993baf3dd837d2bb3bce5f50be3e742d0eb83caed27a30cabcf440d406dbe8f1de897c8c541227dd840541aacdeb21c60b077301556415db,forestfire:https%3A%2F%2Fstorage.googleapis.com%2Fkaggle-data-sets%2F4533424%2F7753317%2Fbundle%2Farchive.zip%3FX-Goog-Algorithm%3DGOOG4-RSA-SHA256%26X-Goog-Credential%3Dgcp-kaggle-com%2540kaggle-161607.iam.gserviceaccount.com%252F20240315%252Fauto%252Fstorage%252Fgoog4_request%26X-Goog-Date%3D20240315T182252Z%26X-Goog-Expires%3D259200%26X-Goog-SignedHeaders%3Dhost%26X-Goog-Signature%3D13f50ef4b6f3b135210beb814bbed3b3c82bbe68d45dca28087cd49312f086e96d55e5f267bda703a07a1b0e5e9cdae2496c5441a2f40b8f46dcfb53a37701b674018cc885150e835704725c80df55fe21a5aa037bf900235cdc277f82401bbefe02a82dc3cb673080366f3efd7cfa60e82ae8749b3f9c1018e81d6c7920104381cc2e189bf2aefb8a96662f425a9748677d6071ab3813ca7955adfd4a86f82e0cba05aba048d46ddeeae40389b05d23585c85bbf74814f0e548310ebcd6c0483091da10e1f35dc1e65fa12427a4d1f669047e8669a4e099e3ca4f885c38177b723495283b7875f365563a4f510be1584fe987ed8a38b38e9a2a72a217080369'

KAGGLE_INPUT_PATH='/kaggle/input'
KAGGLE_WORKING_PATH='/kaggle/working'
KAGGLE_SYMLINK='kaggle'

!umount /kaggle/input/ 2> /dev/null
shutil.rmtree('/kaggle/input', ignore_errors=True)
os.makedirs(KAGGLE_INPUT_PATH, 0o777, exist_ok=True)
os.makedirs(KAGGLE_WORKING_PATH, 0o777, exist_ok=True)

try:
  os.symlink(KAGGLE_INPUT_PATH, os.path.join("..", 'input'), target_is_directory=True)
except FileExistsError:
  pass
try:
  os.symlink(KAGGLE_WORKING_PATH, os.path.join("..", 'working'), target_is_directory=True)
except FileExistsError:
  pass

for data_source_mapping in DATA_SOURCE_MAPPING.split(','):
    directory, download_url_encoded = data_source_mapping.split(':')
    download_url = unquote(download_url_encoded)
    filename = urlparse(download_url).path
    destination_path = os.path.join(KAGGLE_INPUT_PATH, directory)
    try:
        with urlopen(download_url) as fileres, NamedTemporaryFile() as tfile:
            total_length = fileres.headers['content-length']
            print(f'Downloading {directory}, {total_length} bytes compressed')
            dl = 0
            data = fileres.read(CHUNK_SIZE)
            while len(data) > 0:
                dl += len(data)
                tfile.write(data)
                done = int(50 * dl / int(total_length))
                sys.stdout.write(f"\r[{'=' * done}{' ' * (50-done)}] {dl} bytes downloaded")
                sys.stdout.flush()
                data = fileres.read(CHUNK_SIZE)
            if filename.endswith('.zip'):
              with ZipFile(tfile) as zfile:
                zfile.extractall(destination_path)
            else:
              with tarfile.open(tfile.name) as tarfile:
                tarfile.extractall(destination_path)
            print(f'\nDownloaded and uncompressed: {directory}')
    except HTTPError as e:
        print(f'Failed to load (likely expired) {download_url} to path {destination_path}')
        continue
    except OSError as e:
        print(f'Failed to load {download_url} to path {destination_path}')
        continue

print('Data source import complete.')


In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All"
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras import layers, models
from tensorflow.keras.callbacks import LearningRateScheduler
import matplotlib.pyplot as plt
import numpy as np

In [ ]:
train_data_dir = '/kaggle/input/forest-fire-smoke-and-non-fire-image-dataset/FOREST_FIRE_SMOKE_AND_NON_FIRE_DATASET/train'
test_data_dir = '/kaggle/input/forest-fire-smoke-and-non-fire-image-dataset/FOREST_FIRE_SMOKE_AND_NON_FIRE_DATASET/test'

In [ ]:
# Data preprocessing with augmentation and rescaling
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)

test_datagen = ImageDataGenerator(rescale=1./255)

batch_size = 32

train_generator = train_datagen.flow_from_directory(
    train_data_dir,
    target_size=(224, 224),
    batch_size=batch_size,
    class_mode='categorical'
)

test_generator = test_datagen.flow_from_directory(
    test_data_dir,
    target_size=(224, 224),
    batch_size=batch_size,
    class_mode='categorical'
)


Found 32398 images belonging to 3 classes.
Found 10500 images belonging to 3 classes.


In [ ]:
# Build a simple convolutional neural network (CNN) with MobileNetV2 base
base_model = tf.keras.applications.MobileNetV2(input_shape=(224, 224, 3),
                                               include_top=False,
                                               weights='imagenet')
base_model.trainable = False

model = models.Sequential([
    base_model,
    layers.GlobalAveragePooling2D(),
    layers.Dense(128, activation='relu'),
    layers.Dense(3, activation='softmax')
])


9406464/9406464 ━━━━━━━━━━━━━━━━━━━━ 1s 0us/step


In [ ]:
# Compile the model with a learning rate scheduler
initial_learning_rate = 0.001

def scheduler(epoch, lr):
    if epoch < 5:
        return lr
    else:
        return lr * tf.math.exp(-0.1)

lr_scheduler = LearningRateScheduler(scheduler)

model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=initial_learning_rate),
              loss='categorical_crossentropy',
              metrics=['accuracy'])

In [ ]:
# Train the model
epochs = 5
history = model.fit(train_generator,
                    epochs=epochs,
                    validation_data=test_generator,
                    callbacks=[lr_scheduler])

# Evaluate the model on the test set
evaluation_result = model.evaluate(test_generator)
print(f'Test Loss: {evaluation_result[0]}, Test Accuracy: {evaluation_result[1]}')


Epoch 1/5


/opt/conda/lib/python3.10/site-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:122: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


 176/1013 ━━━━━━━━━━━━━━━━━━━━ 17:44 1s/step - accuracy: 0.8453 - loss: 0.3761

In [ ]:
# Evaluate the model on the test set
evaluation_result = model.evaluate(test_generator)
print(f'Test Loss: {evaluation_result[0]}, Test Accuracy: {evaluation_result[1]}')

In [ ]:
from tensorflow.keras.preprocessing import image
from tensorflow.keras.applications.mobilenet_v2 import preprocess_input
import numpy as np

# Load and preprocess the user-provided image
user_image_path = '/path/to/user/image.jpg'  # Replace with the path to the user's image
user_image = image.load_img(user_image_path, target_size=(224, 224))
user_image_array = image.img_to_array(user_image)
user_image_array = np.expand_dims(user_image_array, axis=0)
user_image_array = preprocess_input(user_image_array)

# Make predictions
predictions = model.predict(user_image_array)

# Get the predicted class label
predicted_class = np.argmax(predictions)
predicted_label = list(train_generator.class_indices.keys())[predicted_class]

# Display the results
plt.imshow(user_image)
plt.title(f'Predicted Class: {predicted_label}')
plt.show()